In [9]:
import argparse
from pathlib import Path
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from statsmodels.tsa.holtwinters import ExponentialSmoothing

def infer_file(data_dir: Path, file_arg: str = None) -> Path:
    if file_arg:
        f = Path(file_arg)
        if not f.exists():
            raise FileNotFoundError(f)
        return f
    files = sorted(data_dir.glob("*.csv"), key=lambda p: p.stat().st_mtime, reverse=True)
    if not files:
        raise FileNotFoundError(f"No CSV files in {data_dir}")
    return files[0]

def load_series(path: Path, y_col: str = "y"):
    df = pd.read_csv(path)
    if y_col not in df.columns:
        raise ValueError(f"Column '{y_col}' not found in {path}")
    series = df[y_col].copy()
    # If there's a 't' column and it's datetime-like, try to set index
    if "t" in df.columns:
        try:
            idx = pd.to_datetime(df["t"])
            series.index = idx
        except Exception:
            series.index = pd.RangeIndex(start=0, stop=len(series), step=1)
    else:
        series.index = pd.RangeIndex(start=0, stop=len(series), step=1)
    return series, df

def make_future_index(index, h):
    if isinstance(index, pd.DatetimeIndex):
        freq = index.freq or pd.infer_freq(index)
        if freq is None:
            # fallback to daily
            freq = "D"
        return pd.date_range(start=index[-1] + pd.tseries.frequencies.to_offset(freq), periods=h, freq=freq)
    else:
        return np.arange(len(index), len(index) + h)

In [10]:
p = argparse.ArgumentParser(description="Holt-Winters forecasting (Exponential Smoothing)")
p.add_argument("--file", type=str, default=None, help="CSV file in data/ (if omitted, latest CSV is used)")
p.add_argument("--horizon", type=int, default=24, help="Forecast horizon (steps)")
p.add_argument("--seasonal_periods", type=int, default=None, help="Seasonal period (e.g. 12)")
p.add_argument("--trend", type=str, choices=["add", "mul", "none"], default="add")
p.add_argument("--seasonal", type=str, choices=["add", "mul", "none"], default="add")
p.add_argument("--ycol", type=str, default="y", help="Name of target column in CSV")

import sys
# в ноутбуке не парсим аргументы из командной строки
if "ipykernel" in sys.modules:
    args = p.parse_args([])  # использовать значения по умолчанию
else:
    args = p.parse_args()

# корректное формирование папки data (относительно рабочего каталога)
project_root = Path.cwd().resolve().parent  # предполагается структура repo/.../notebook
data_dir = project_root / "data"
out_dir = project_root / "output"
data_dir.mkdir(parents=True, exist_ok=True)

csv_path = data_dir / "ts_1000_1_02.csv"
series, raw_df = load_series(csv_path, args.ycol)

trend = None if args.trend == "none" else args.trend
seasonal = None if args.seasonal == "none" else args.seasonal
seasonal_periods = args.seasonal_periods

# If seasonal_periods not provided but seasonal requested, try to infer simple default
if seasonal and seasonal_periods is None:
    seasonal_periods = 12

In [11]:
model = ExponentialSmoothing(
    series,
    trend=trend,
    seasonal=seasonal,
    seasonal_periods=seasonal_periods,
    initialization_method="estimated"
)
fitted = model.fit(optimized=True)

/opt/conda/lib/python3.12/site-packages/statsmodels/tsa/base/tsa_model.py:473: ValueWarning: No frequency information was provided, so inferred frequency ns will be used.
  self._init_dates(dates, freq)


In [ ]:
fcast = fitted.forecast(args.horizon)
f_index = make_future_index(series.index, args.horizon)
fcast.index = f_index

# Save forecast CSV in format: index, value (index as number like in source)
out_name = f"{csv_path.stem}_hw_h{args.horizon}_s{seasonal_periods if seasonal_periods else 0}.csv"
out_path = out_dir / out_name

# If forecast index is datetime, convert to numeric positions to match original numeric index
if isinstance(fcast.index, pd.DatetimeIndex):
    idx_vals = np.arange(len(series), len(series) + len(fcast))
else:
    idx_vals = fcast.index.astype(int)

df_out = pd.DataFrame({"index": idx_vals, "value": fcast.values})
df_out.to_csv(out_path, index=False)

In [13]:

# Save plot
plt.figure(figsize=(10, 5))
plt.plot(series.index, series.values, label="y", marker="o")
plt.plot(series.index, fitted.fittedvalues, label="fitted", alpha=0.7)
plt.plot(fcast.index, fcast.values, label="forecast", marker="o")
plt.legend()
plt.title(f"Holt-Winters forecast ({csv_path.name})")
plot_path = out_dir / f"{csv_path.stem}_hw_plot_h{args.horizon}.png"
plt.tight_layout()
plt.savefig(plot_path)
plt.close()

print(f"Loaded: {csv_path}")
print(f"Forecast saved: {out_path}")
print(f"Plot saved: {plot_path}")

Loaded: /home/jovyan/work/Time-series-forecasting-Holt-Winters-model/data/ts_1000_1_02.csv
Forecast saved: /home/jovyan/work/Time-series-forecasting-Holt-Winters-model/output/ts_1000_1_02_hw_h24_s12.csv
Plot saved: /home/jovyan/work/Time-series-forecasting-Holt-Winters-model/output/ts_1000_1_02_hw_plot_h24.png
